In [1]:
import pandas as pd
import numpy as np
from math import sqrt

from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from sequence_distance_matrix import *


db = pd.read_csv('dataset/db_modelli_smiles_pubchem.csv').drop(columns = 'Unnamed: 0')
# codifica
db['conc1_mean'] = np.where(db['conc1_mean'].values > 1, 1, 0)

db = db.drop(columns = 'test_cas')

# Ordinal Encoding
encoder = OrdinalEncoder(dtype = int)

encoder.fit(db[['species', 'conc1_type', 'exposure_type', 'class', 'tax_order', 'family', 'genus']])

db[['species', 'conc1_type', 'exposure_type', 'class', 'tax_order', 'family', 'genus']] = encoder.transform(
    db[['species', 'conc1_type', 'exposure_type', 'class', 'tax_order', 'family', 'genus']])+1

# Divido il dataset completo tra dati e target
X = db.drop(columns = 'conc1_mean')
y = db['conc1_mean'].values

# splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# ALLA FINE DELLA CROSS-VALIDATION SI PROCEDE COME PER UN MODELLO SINGOLO...VEDERE KNN--BINARY.ipynb
# X_try = X_train.append(X_test)

# # tengo traccia della lunghezza del train set
# len_X_train = len(X_train) 





# KNN 1+2+3 -- CV

In [12]:
# due ore circa--> con 10 alpha
c = [1,0]
pub = np.logspace(-3, 0, 30) 

categorical = ['ring_number', "exposure_type", "conc1_type","species",'tripleBond', 'obs_duration_mean', 'doubleBond',
    'alone_atom_number', 'class', 'tax_order', 'family', 'genus', 'oh_count']

non_categorical =[ 'atom_number', 'bonds_number', 'Mol', 'MorganDensity', 'LogP']

best_acc, best_alpha, best_k, best_leaf = cv_params(X_train, y_train, categorical, non_categorical,
                                                    sequence_pub = pub, choice = c)

START...
Computing Basic Matrix: Hamming 1 and Euclidean 2...

 Adding Hamming 3 (Pubchem2d)... alpha = 0.001
Start CV...
New best params found! alpha:0.001, k:1, leaf:30, acc:0.892288
New best params found! alpha:0.001, k:1, leaf:40, acc:0.8961279999999998
New best params found! alpha:0.001, k:1, leaf:60, acc:0.8965759999999999

 Adding Hamming 3 (Pubchem2d)... alpha = 0.0012689610031679222
Start CV...

 Adding Hamming 3 (Pubchem2d)... alpha = 0.0016102620275609393
Start CV...
New best params found! alpha:0.0016102620275609393, k:1, leaf:100, acc:0.8965760000000002

 Adding Hamming 3 (Pubchem2d)... alpha = 0.0020433597178569417
Start CV...

 Adding Hamming 3 (Pubchem2d)... alpha = 0.002592943797404667
Start CV...

 Adding Hamming 3 (Pubchem2d)... alpha = 0.0032903445623126675
Start CV...

 Adding Hamming 3 (Pubchem2d)... alpha = 0.0041753189365604
Start CV...

 Adding Hamming 3 (Pubchem2d)... alpha = 0.005298316906283708
Start CV...

 Adding Hamming 3 (Pubchem2d)... alpha = 0.00672335

In [2]:
from mem_distance_matrix import *

categorical = ['ring_number', "exposure_type", "conc1_type","species",'tripleBond', 'obs_duration_mean', 'doubleBond',
    'alone_atom_number', 'class', 'tax_order', 'family', 'genus', 'oh_count']

non_categorical =[ 'atom_number', 'bonds_number', 'Mol', 'MorganDensity', 'LogP']

best_k = 1
best_alpha = 0.2395026619987486
best_leaf = 90


len_X_train = len(X_train) 

X_try = X_train.append(X_test)

X_train_new, X_test_new = train_test_distances(X_try, len_X_train, categorical, non_categorical,
                                               alphas = [0.0016102620275609393, 1,
                                                         best_alpha, 0], choice = [1,0])

neigh123 = KNeighborsClassifier(metric = 'precomputed', n_neighbors = best_k, leaf_size = best_leaf)
neigh123.fit(X_train_new, y_train.ravel())
y_pred = neigh123.predict(X_test_new)
print('Model: {}, \n Accuracy: {},\n RMSE: {}'.format(neigh123,
                                                      accuracy_score(y_test, y_pred),
                                                      sqrt(mean_squared_error(y_test, y_pred))))

Start...
You choose Hamming 1, Euclidean 2 and Hamming on pubchem2d 3...
Start Hamming su categorical...
Start Euclidean...
Start Hamming su Pubchem2d...
Combining...
Parametri: Hamming 1 --> 0.0016102620275609393, Euclidean 2 --> 1, Pubchem2d 3 --> 0.2395026619987486
...FINISH
Model: KNeighborsClassifier(algorithm='auto', leaf_size=90, metric='precomputed',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                     weights='uniform'), 
 Accuracy: 0.8294140574249708,
 RMSE: 0.4130205110827176
